## WMS Agent with ADK

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/wms-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Fwms-agent-mcp.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/wms-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [8]:
%pip install google-adk -q
%pip install litellm -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [9]:
import os
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams

### Set Auth

In [10]:
#os.environ["GOOGLE_API_KEY"] = "" # <--- REPLACE
CLOUD_PROJECT = "cymbal-ai" # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"]= CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"]="us-central1"

### Get Apigee MCP Tools

In [11]:
async def get_wms_tools_async():
  """Gets tools from the OMS MCP Server."""
  print("Attempting to connect to WMS MCP server...")
  
  mcp_base_url = "https://34.107.220.83.nip.io/wms-mcp-proxy/sse"  # @param {type:"string"}
  if not mcp_base_url:
    raise ValueError("Variable MCP_BASE_URL is not set.")

  client_id = "Kcf6MeXh6WaLIqPTAEDRljLGZakVm81OL1ohsmt81u6HoXpb"  # @param {type:"string"}

  if not client_id:
    raise ValueError("Environment variable OMS_TOOLS_API_KEY is not set.")

  tools, exit_stack = await MCPToolset.from_server(
      connection_params=SseServerParams(
          url=mcp_base_url,
          headers={"x-api-key": client_id},
      )
  )
  print("MCP Customer Toolset created successfully.")
  return tools, exit_stack

### Initialize Agent

In [12]:
async def get_wms_agent_async():
    """Creates a Customer Agent"""
    tools, exit_stack = await get_wms_tools_async()
    print(f"Fetched {len(tools)} WMS tools from MCP server.")
    customer_agent = Agent(
        model="gemini-2.5-pro-preview-05-06",
        name="customer_agent",
        description="Central authority for all warehouse and fulfillment matters",
        instruction="""
        - You are esponsible for managing and executing the physical fulfillment process once an order is released by the Order Management System (OMS). You act as the central nervous system for warehouse operations, taking new order details and orchestrating the picking, packing, and shipping stages. You ensure a seamless flow of information by updating the OMS with real-time order status changes (e.g., 'Awaiting Fulfillment', 'Picking', 'Packed', 'Shipped') and by directly interfacing with shipping carrier systems to generate labels, schedule pickups, and retrieve tracking information. Your primary objective is to ensure orders are processed accurately, efficiently, and are dispatched to customers in a timely manner.
        - Use the tools at your disposal to:
            - Create new orders in the warehouse management system (WMS) based on order details from the OMS agent
            - Receive new order details from the OMS agent for picking and packing Updating order status back to the WMS (e.g., 'Awaiting Fulfillment', 'Picking', 'Packed') 
            - Faciliate shipping operations, including generating shipping labels, scheduling pickups, and retrieving tracking information
        """,
        tools=tools
    )
    return customer_agent, exit_stack

### Build Agent Runtime

In [13]:
async def async_main():
  session_service = InMemorySessionService()
  artifacts_service = InMemoryArtifactService()

  session = session_service.create_session(
      state={}, app_name='wms_app', user_id='user_fs'
  )

  query = "Can you generate a shipping label?"
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent, exit_stack = await get_wms_agent_async()

  runner = Runner(
      app_name='wms_app',
      agent=root_agent,
      artifact_service=artifacts_service,
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=session.user_id, new_message=content
  )

  async for event in events_async:
    print(f"Event received: {event}")

  # Crucial Cleanup: Ensure the MCP server process connection is closed.
  print("Closing MCP server connection...")
  await exit_stack.aclose()
  print("Cleanup complete.")

### Execute Agent

In [14]:
await async_main()

User Query: 'Can you generate a shipping label?'
Attempting to connect to WMS MCP server...


  + Exception Group Traceback (most recent call last):
  |   File "/Users/gonzalezruben/Documents/apigee-adk-workshop/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3668, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_64512/2229161767.py", line 1, in <module>
  |     await async_main()
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_64512/438978371.py", line 13, in async_main
  |     root_agent, exit_stack = await get_wms_agent_async()
  |                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_64512/2732294698.py", line 3, in get_wms_agent_async
  |     tools, exit_stack = await get_wms_tools_async()
  |                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/nx/fvpj4ftj0qg1pn5dnjrm2q4h00jrr5/T/ipykernel_64512/3836312361.py", line 14, in get_wms